# Analyser les données ouvertes du patrimoine

D’importantes bases de connaissance sont disponibles sous licence libre, transformant la recherche.
Avec la possibilité de mobiliser les méthodes computationnelles sur un volume considérable de données inédites.
À condition bien sûr de savoir accéder aux données, les traiter et les analyser.

Ce petit démonstrateur est une initiation aux méthodes de collecte, de traitement et d’analyse des données patrimoniales.

Nous découvrirons :

- les méthodes d’accès aux données via des APIs, avec la librairie Python `requests` ;
- l’écosystème des données ouvertes partagées par la BnF, Gallica en particulier, et DBPedia ;
- une méthode pour la reconnaissance et le liage des entités nommées (*Entity linking*) pour enrichir les données collectées ;
- comment construire une chaîne de traitement complète de *geoparsing* dans des textes (issus de Gallica) ;
- une méthode de visualisation (cartographique) des résultats ;
- comment une carte de chaleur avec la bibliothèque `folium`.

Ce faisant, vous comprendrez mieux le fonctionnement du Web (HTTP), découvrirez les formats de données structurés (CSV, JSON, XML) et aurez une expérience pratique du langage de programmation Python.

Ce tutoriel reprend largement le cours de **Bertrand Dumenieu** (EHESS) dédié à l'expérimentation du traitement automatique du langage naturel (TALN) grâce à un modèle d'apprentissage profond. Un grand merci à lui pour le partage de ces éléments.

## Automatiser la collecte de données

### Gallica

Librement accessible depuis 1997, [Gallica](https://gallica.bnf.fr/) est la bibliothèque numérique de la Bibliothèque nationale de France et de ses partenaires.

- accès libre et gratuit à 10 millions de documents numérisés
- de toutes époques et de tous supports (manuscrits, journaux, etc.)
- une application de recherche et de consultation : [https://gallica.bnf.fr/accueil/fr/html/accueil-fr](https://gallica.bnf.fr/accueil/fr/html/accueil-fr)
- des APIs : [https://api.bnf.fr/fr/recherche?q=recherche&f%5B0%5D=categories%3A4&f%5B1%5D=sources%3A197](https://api.bnf.fr/fr/recherche?q=recherche&f%5B0%5D=categories%3A4&f%5B1%5D=sources%3A197)

Une API (*Application Programming Interface*) est un moyen pour des programmes informatiques de communiquer entre eux.
Contrairement à une interface utilisateur, qui relie un ordinateur à une personne, une API relie des ordinateurs ou des logiciels entre eux. Elle n'est pas destinée à être utilisée directement par une personne (l'utilisateur final) autre qu'un programmeur informatique qui l'incorpore dans le logiciel.

- Une API permet à un ordinateur de demander des informations à un autre ordinateur via l'internet.
- Les points d'accès aux données et le format de la réponse sont normalisés conformément à une spécification.


### Gallica Search API (XML)


La BnF a fait un travail impressionnant de mise à disposition de ces ressources ouvertes, via différentes API qui permettent d'interagir avec différents services : métadonnées, données textuelles, mais aussi recueil de données iconographiques.

Ces [API de Gallica sont documentées](https://api.bnf.fr/recherche?f[0]=sources:197) de manière exemplaire, avec une attention particulière portée aux besoins des chercheurs.

**L’API de recherche (SRU) permet d'effectuer des recherches dans la collection numérique de Gallica:**


- **Documentation** : [https://api.bnf.fr/fr/api-gallica-de-recherche](https://api.bnf.fr/fr/api-gallica-de-recherche)
- **Service de recherche Gallica SRU** : https://api.bnf.fr/fr/api-gallica-de-recherche#scroll-nav__3

SRU (Search/Retrieval via URL) est un **protocole standard d'échange de métadonnées** adapté aux besoins des catalogues de bibliothèques. En d'autres termes, les bibliothèques ont collaboré pour définir un moyen normalisé d'exposer leur catalogue en tant que service web.

La norme SRU spécifie **comment interroger** le serveur qui expose les données du catalogue et **comment formater la réponse**.

Lire : https://bibliotheques.wordpress.com/2017/10/27/papa-cest-quoi-un-sru/

**En utilisant l'API de recherche, commençons par essayer de collecter automatiquement les données relatives aux romans de Jules Verne.** 



### CQL Query

La base : `https://gallica.bnf.fr/SRU?version=1.2&operation=searchRetrieve&query={CQL_QUERY}`


[CQL, Contextual Query Language](https://www.loc.gov/standards/sru/cql/), est un langage formel pour adresser des requêtes aux systèmes de recherche d'informations tels que les index Web, les catalogues bibliographiques et les informations sur les collections de musées. L'objectif de la conception est que les requêtes soient lisibles et facile à rédiger par l'homme, et que le langage soit intuitif tout en conservant l'expressivité de langages plus complexes.


Quelques exemples pour vous aider à comprendre.


**Les documents mentionnant "Jules Verne" dans Gallica** (>18752 records…):

- `query=gallica all "Jules Verne"`
- [https://gallica.bnf.fr/SRU?version=1.2&operation=searchRetrieve&**query=gallica all "Jules Verne"**](https://gallica.bnf.fr/SRU?version=1.2&operation=searchRetrieve&query=gallica%20all%20%22Jules%20Verne%22)

**Les documents dont "Jules Verne" est l’auteur dans Gallica** (>400 records):  

- `query=dc.creator all "jules verne"`
- [https://gallica.bnf.fr/SRU?version=1.2&operation=searchRetrieve&**query=dc.creator all "Jules Verne"**](https://gallica.bnf.fr/SRU?version=1.2&operation=searchRetrieve&query=dc.creator%20all%20%22Jules%20Verne%22)

**Les livres (monographs) en français dont "Jules Verne" est l’auteur dans Gallica** (>290 records):

- `query=dc.creator all "jules verne"&filter=dc.type all "monographie" and dc.language all "fre"`
- [https://gallica.bnf.fr/SRU?version=1.2&operation=searchRetrieve&**query=dc.creator all "jules verne"&filter=dc.type all "monographie" and dc.language all "fre"**](https://gallica.bnf.fr/SRU?version=1.2&operation=searchRetrieve&query=dc.creator%20all%20%22jules%20verne%22&filter=dc.type%20all%20%22monographie%22%20and%20dc.language%20all%20%22fre%22)


On peut **trier les résultats** selon différents critères.

Les résultats peuvent être triés selon la qualité de l’OCR de la couche texte : `ocr.quality/sort.descending`:

[https://gallica.bnf.fr/SRU?version=1.2&operation=searchRetrieve&query=dc.creator all "jules verne" sortby ocr.quality/sort.descending&filter=dc.type all "monographie" and dc.language all "fre"](https://gallica.bnf.fr/SRU?version=1.2&operation=searchRetrieve&query=dc.creator%20all%20%22jules%20verne%22%20sortby%20ocr.quality/sort.descending&filter=dc.type%20all%20%22monographie%22%20and%20dc.language%20all%20%22fre%22)

Il existe de nombreux autres paramètres :
 
- `startRecord` : l'indice du système de pagination, entre 1 et le nombre maximum de résultats retournés par la requête
- `maximumRecords` : le nombre de résultats retournés par le service (de 0 à un maximum de 50). Par défaut, si ce paramètre n'est pas présent, la valeur est de 15.


**Dans Gallica, 3 (`&maximumRecords=3`) monographies (`dc.type all "monographie"`) en français (`dc.language all "fre"`) de 1966 (`dc.date="1966"`) mentionnant le "Festival Mondial des Arts Nègres"** :

- `query=gallica all "Festival Mondial des Arts Nègres" sortby ocr.quality/sort.descending&filter=dc.type all "monographie" and dc.language all "fre" and dc.date="1966"&maximumRecords=3`
- [https://gallica.bnf.fr/SRU?version=1.2&operation=searchRetrieve&query=gallica all "Festival Mondial des Arts Nègres" sortby ocr.quality/sort.descending&filter=dc.type all "monographie" and dc.language all "fre" and dc.date="1966"&maximumRecords=3](https://gallica.bnf.fr/SRU?version=1.2&operation=searchRetrieve&query=gallica%20all%20%22Festival%20Mondial%20des%20Arts%20N%C3%A8gres%22%20sortby%20ocr.quality/sort.descending&filter=dc.type%20all%20%22monographie%22%20and%20dc.language%20all%20%22fre%22%20and%20dc.date=%221966%22&maximumRecords=3)

Dans Gallica, les revues (`dc.type all "fascicule"`) en français postérieures à 1976 (`dc.date >= "1976"`) mentionnant le "FESTAC" :


- `&query=gallica all "FESTAC" sortby ocr.quality/sort.descending&filter=dc.type all "fascicule" and dc.language all "fre" and dc.date >= "1976"`
- [https://gallica.bnf.fr/SRU?version=1.2&operation=searchRetrieve&query=gallica all "FESTAC" sortby ocr.quality/sort.descending&filter=dc.type all "fascicule" and dc.language all "fre" and dc.date >= "1976"](https://gallica.bnf.fr/SRU?version=1.2&operation=searchRetrieve&query=gallica%20all%20%22FESTAC%22%20sortby%20ocr.quality/sort.descending&filter=dc.type%20all%20%22fascicule%22%20and%20dc.language%20all%20%22fre%22%20and%20dc.date%20%3E=%20%221976%22)

Dans le cas des revues, on obtient l’ARK de la revue et non directement celui de la publication contenant l’expression recherchée…

In [ ]:
TROUVER AUTRE EX
url = 'https://gallica.bnf.fr/SRU?version=1.2&operation=searchRetrieve&query=gallica%20all%20%22FESTAC%22%20sortby%20ocr.quality/sort.descending&filter=dc.type%20all%20%22fascicule%22%20and%20dc.language%20all%20%22fre%22%20and%20dc.date%20%3E=%20%221976%22'
response = requests.get(url)
print(response.text)


### Requêtes et réponses HTTP (requests)

Pour accéder par programme aux données textuelles attachées à chaque URL, nous pouvons utiliser une bibliothèque Python appelée [requests](https://requests.readthedocs.io/en/master/).

Une ULR ([Uniform Resource Locator](https://en.wikipedia.org/wiki/URL))est une référence à une ressource web qui spécifie son emplacement sur un réseau informatique et un mécanisme pour la récupérer.

Chaque URL HTTP est conforme à la syntaxe d'un URI générique. La syntaxe générique de l'URI se compose de 6 éléments organisés hiérarchiquement :

`http://www.domain.com:80/path/to/myfile.html?key1=value1&key2=value2#anchor_in_doc`

URLS parts:

- **protocol**: `http://` or `https://`.
- **domain name**: `www.domain.com` – au lieu d'un nom de domaine, vous pouvez utiliser une adresse IP.
- **port**: `:80` - indique la "porte" technique à utiliser pour accéder aux ressources du serveur. Ce fragment est généralement absent, car le navigateur utilise les ports standards associés aux protocoles (80 pour HTTP, 443 pour HTTPS).
- **path**: `/path/to/myfile.html` –chemin, sur le serveur web, vers la ressource. Dans les premiers temps du Web, ce chemin correspondait souvent à un chemin « physique » existant sur le serveur. Aujourd'hui, ce chemin n'est qu'une abstraction gérée par le serveur web, et ne correspond plus à une réalité "physique".
- **parameters**: `?key1=value1&key2=value2` – construit comme une liste de paires clé/valeur séparées par une esperluette.
- **anchor**: `#anchor_in_doc` – pointe vers un endroit donné de la ressource.

Lorsque vous tapez une URL dans la barre d'adresse de votre navigateur, vous envoyez une **demande** HTTP pour une page web, et le serveur qui stocke cette page web renverra une **réponse**.

<img src="./img/http.png" width="600px">


Une requête HTTP dont les paramètres sont passés via l'URL est dite de type GET.

Avec le librairie requests, on peut très facilement exécuter ce genre de requêtes et récupérer une réponse grâce à la fonction `requests.get(...)` :

In [ ]:
import requests
url = "https://gallica.bnf.fr/SRU?version=1.2&operation=searchRetrieve&query=dc.creator%20all%20%22jules%20verne%22%20sortby%20ocr.quality/sort.descending&filter=dc.type%20all%20%22monographie%22%20and%20dc.language%20all%20%22fre%22&maximumRecords=5"
response = requests.get(url)
response.raise_for_status() # Appeler raise_for_status() après une requête GET est une bonne pratique : cela permet de lever automatiquement une exception si la requête a échoué.
print(type(response), response)

`requests.get()` renvoie donc un objet Python de type `requests.models.Response`, qui quand on l'affiche donne le code HTTP retour, ici 200, qui signifie que [tout s'est bien passé ](https://developer.mozilla.org/fr/docs/Web/HTTP/Reference/Status/200).

On peut alors récupérer le contenu de la réponse de plusieurs manières :

In [ ]:
response.headers['Content-Type']

Pour accéder à l’information et l’extraire nous avons besoin de ***parser* la donnée formatée en XML**!

**[The ElementTree XML API](https://docs.python.org/3/library/xml.etree.elementtree.html)**. Le module `xml.etree.ElementTree` (`ET` en abrégé) implémente une API simple et efficace pour l'analyse et la création de données XML.

XML est un format de données hiérarchique, et la manière la plus intuitive de le représenter est un arbre. **ET dispose de deux classes** à cet effet :

- `ElementTree` représente l'ensemble du document XML sous forme d'arbre
- `Element` représente un nœud unique dans cet arbre.

Les interactions avec l'ensemble du document (lecture et écriture dans les fichiers) se font généralement au niveau de l'`ElementTree`. Les interactions avec un seul élément XML et ses sous-éléments se font au niveau de l'`Element`.

**[fromstring() analyse le XML à partir d'une chaîne directement dans un `Element`](https://docs.python.org/3/library/xml.etree.elementtree.html#xml.etree.ElementTree.fromstring)** - stocké ci-dessous dans la variable `root` qui est l'élément racine de l'arbre analysé.

Ensuite, `Element` a quelques méthodes utiles qui aident à itérer récursivement sur tous les sous-arbres en dessous de lui (ses enfants, leurs enfants, et ainsi de suite). Par exemple, `Element.iter()`.

**[La méthode iter()](https://docs.python.org/3/library/xml.etree.elementtree.html#xml.etree.ElementTree.Element.iter)** crée un itérateur d'arbre avec l'élément courant comme racine. L'itérateur parcourt cet élément et tous les éléments situés en dessous, dans l'ordre du document (profondeur en premier).


In [ ]:
import xml.etree.ElementTree as ET
root = ET.fromstring(response.content)
for child in root.iter('*'):
    print(child.tag)

### Extraction des métadonnées (Search API)

Nous voyons que tous les éléments de la réponse sont accessibles, selon leur espace de noms : `{http://purl.org/dc/elements/1.1/}creator` 
L'élément `creator` est déclaré pour l'espace de nom Dublin Core (`http://purl.org/dc/elements/1.1/`).

En utilisant une boucle `for`, chaque `record` est lu, et la méthode `Element.findall()` est utilisée pour extraire les métadonnées du Dublin Core :

- `identifier`
- `date`
- `title`
- `creator`

NB. La méthode [`Element.findall()`](https://docs.python.org/3/library/xml.etree.elementtree.html#xml.etree.ElementTree.Element.findall) ne trouve que les éléments avec une balise qui sont des enfants directs de l'élément courant.


In [ ]:
import pandas as pd

def get_gallica_metadata(search_api_url, dc_elements_array):
    response = requests.get(search_api_url)
    root = ET.fromstring(response.content)
    metadata = []
    for record in root.iter('{http://www.loc.gov/zing/srw/}record'):
        record_meta_dic = {}
        for dc_el in dc_elements_array:
            if record.findall('{http://www.loc.gov/zing/srw/}recordData/{http://www.openarchives.org/OAI/2.0/oai_dc/}dc/{http://purl.org/dc/elements/1.1/}'+dc_el):
                record_meta_dic[dc_el] = record.findall('{http://www.loc.gov/zing/srw/}recordData/{http://www.openarchives.org/OAI/2.0/oai_dc/}dc/{http://purl.org/dc/elements/1.1/}'+dc_el)[0].text
            else :
                record_meta_dic[dc_el] = 'NaN'
        # non DC metadata
        record_meta_dic['ocr_quality'] = record.findall('{http://www.loc.gov/zing/srw/}extraRecordData/nqamoyen')[0].text if record.findall('{http://www.loc.gov/zing/srw/}extraRecordData/nqamoyen')[0].text else 'NaN'
        metadata.append(record_meta_dic)
    metadata_df = pd.DataFrame(metadata)
    return metadata_df

In [ ]:
#url = "https://gallica.bnf.fr/SRU?version=1.2&operation=searchRetrieve&query=dc.creator%20all%20%22jules%20verne%22%20sortby%20ocr.quality/sort.descending&filter=dc.type%20all%20%22monographie%22%20and%20dc.language%20all%20%22fre%22&startRecord=1&maximumRecords=5"
books_df = get_gallica_metadata(url, ['creator', 'identifier', 'title'])
books_df


### Extraction du plein texte (Document API)

Documentation : https://api.bnf.fr/fr/api-document-de-gallica

Pour document trouvé via l'API de recherche ou l'interface Gallica, l'API Document peut être utilisée pour récupérer les métadonnées nécessaires à l'utilisation des ressources numériques du document, y compris :

- les informations bibliographiques
- les résultats de recherche
- **le texte (texte brut / OCR)**

Ainsi, pour un identifiant ark, si le document a été "OCRisé", il est toujours possible de récupérer tout le texte.

C'est vraiment très simple !

Lorsqu'un document est indexé en texte intégral, il est possible d'obtenir ce texte en utilisant le qualificateur `textBrut` :  
https://gallica.bnf.fr/{ark}.texteBrut

Pour l’exemple, nous travaillerons sur l’édition française du Bulletin d'information du Conseil international des musées (parution de décembre 1964), première mention du Festival mondial des arts nègres :  
https://gallica.bnf.fr/ark:/12148/bpt6k65591428



In [ ]:
ark_url = 'https://gallica.bnf.fr/ark:/12148/bpt6k65591428'
text_url = f"{ark_url}.texteBrut"
response = requests.get(text_url)
print(response.text[0:2000])

Que trouve-t-on ? Étonnamment, le texte n’est pas réellement disponible en texte brut, mais est formaté en HTML…
On pourrait se contenter de supprimer les balises. Mais il y a mieux à faire.

En examinant attentivement le document on s’aperçoit que le texte débute après la première balise HTML `<hr/>`.  
Grâce à la librairie [BeautifulSoup](https://pypi.org/project/beautifulsoup4/), nous pouvons extraire le seul texte du document.

In [ ]:
from bs4 import BeautifulSoup

In [ ]:
def get_doc_text(ark_url):
    response = requests.get(ark_url+'.texteBrut')
    document = BeautifulSoup(response.text, "html.parser")
    first_hr_tag = document.select('hr')[0]
    text = ''
    for p in first_hr_tag.find_all_next('p'):
        text += p.text + '\n'
    return text

In [ ]:
print(get_doc_text(ark_url)[0:2000])

### Enregistrer le texte dans un fichier

Si on souhaitera le plus souvent stocker les données dans un Dataframe, on a souvent besoin d’enregistrer les données en local…


In [ ]:
file_name = ark_url[34:] # on ne retient que le suffixe de l’ark pour le nom du fichier
with open(f'docs/{file_name}.txt', 'a') as file:
    file.write(get_doc_text(ark_url))

**Pour nous prémunir des problèmes de réseau, nous allons travailler avec ce même document enregistré : `./docs/bpt6k65591428`.**

## Reconnaissance des entités nommées avec spaCy

SpaCy est une bibliothèque logicielle Python pour le traitement automatique de nombreuses langues. Il s'agit d'une boîte à outils essentielle pour l'analyse computationnelle de corpus de textes.

À lire :

- https://spacy.io/
- Avanced NLP with Spacy : https://course.spacy.io/en/
- https://github.com/mchesterkadwell/named-entity-recognition/blob/main/1-basic-text-mining-concepts.ipynb
- for NER : https://melaniewalsh.github.io/Intro-Cultural-Analytics/05-Text-Analysis/12-Named-Entity-Recognition.html
- for Text-Mining (basics) : https://github.com/mchesterkadwell/named-entity-recognition/blob/main/1-basic-text-mining-concepts.ipynb 



In [ ]:
# Install and import Spacy
!pip install -U spacy

In [ ]:
import spacy

### Pipelines de traitement du langage et modèles entraînés

[https://spacy.io/usage/processing-pipelines](https://spacy.io/usage/processing-pipelines)

Lorsque vous appelez nlp sur un texte, spaCy commence par tokeniser le texte pour produire un [objet Doc](https://spacy.io/api/doc). Le document est ensuite traité en plusieurs étapes (c'est-à-dire le pipeline de traitement). Chaque composant du pipeline renvoie le document traité, qui est ensuite transmis au composant suivant.

![spacy_pipeline](img/spacy_pipeline.svg)

Les capacités d'un pipeline de traitement dépendent toujours des composants, de leurs modèles et de la manière dont ils ont été formés. Par exemple, un pipeline de reconnaissance d'entités nommées doit inclure un composant de reconnaissance d'entités nommées formé.

Il convient de se référer à la documentation des modèles mis à disposition : [https://spacy.io/models](https://spacy.io/models)

#### Installation et importation d'un pipeline 

In [ ]:
#!conda install -c conda-forge spacy-model-fr_core_news_md

In [ ]:
import fr_core_news_md
nlp = fr_core_news_md.load()

In [ ]:
type(nlp)

In [ ]:
print(nlp.pipe_names)

### Reconnaissance des entités nommées¶

La reconnaissance des entités nommées (NER) est le processus qui consiste à localiser les entités nommées et à les classer dans des catégories prédéfinies, telles que les noms de personnes, les lieux, les organisations.

spaCy possède la propriété .ents sur les objets Doc. Vous pouvez l'utiliser pour extraire des entités nommées :

In [ ]:
from IPython import display
from spacy import displacy

with open("docs/bpt6k65591428.txt", "r") as file:
    texte = file.read()

doc = nlp(texte)
# ⚠️ ATTENTION, la sortie de la cellule suivante est très longue, n'hésitez pas à commenter la ligne suivante une fois que vous avez vu le résultat
display.HTML(displacy.render(doc, style="ent", jupyter=False, page=True))


In [ ]:
with open("docs/bpt6k65591428.txt", "r") as file:
    texte = file.read()

doc = nlp(texte)

for ent in doc[3500:3550].ents:
    print(
        f"""
        {ent.text = }
        {ent.label_ = }
        type = {spacy.explain(ent.label_)}"""
    )

Dans l'exemple ci-dessus, `ent` est un [Span object] (https://spacy.io/api/span) avec divers attributs :

- `.text` donne la représentation textuelle Unicode (la chaîne) de l'entité.
- `.label_` donne l'étiquette de l'entité.
- `.start_char` indique le décalage de caractère pour le début de l'entité.
- `.end_char` indique le décalage de caractère pour la fin de l'entité.

`spacy.explain()` donne des détails descriptifs sur chaque étiquette d'entité.

Comptage des lieux : pour une meilleure compréhension, nous proposons 2 méthodes :

- La première avec une boucle `for`, pour comprendre comment lire la liste des entités.
- La seconde (List comprehension) est plus "pythonique".

In [ ]:
from collections import Counter
# method 1:
places = []
for named_entity in doc.ents:
    if named_entity.label_ == "LOC":
        places.append(named_entity.text)

print(Counter(places).most_common(10))

In [ ]:
# method 2 (pythonic):
places = [
    entity.text
    for entity in doc.ents
    if entity.label_ == 'LOC'
]
places_df = pd.DataFrame(Counter(places).most_common(20), columns=['place', 'count'])
places_df.iloc[0:10]

Vous pouvez également utiliser displaCy pour visualiser ces entités. Ici, nous ne visualisons que quelques phrases (`list(doc.sents)[n:m]`), mais il est bien sûr possible d'annoter le texte entier (`doc`).

In [ ]:
from spacy import displacy
displacy.render(list(doc.sents)[1000:1100], style='ent', jupyter=True)

On constate surtout que le modèle est assez mauvais par défaut. Plusieurs raisons

Pour l’exercice nous avons préparé un modèle plus efficace (voir `ner/model-best`).

Pour comprendre comment ce modèle a été entraîné (avec les données partagées par le BnF), lire le [cours de Bertrand Duménieu](https://github.com/HueyNemud/enc-tnah-atelierspython-nlp/tree/main/partie_1).

<style>
    .renbox {
        width: 100%;
        height: 100%;
        padding: 2px;
        margin: 3px;
        border-radius: 3px;
        box-shadow: 0 0 10px rgba(0, 0, 0, 0.1);
        font-weight: bolder;
        background-color: #8b8c8b;
    }
    .per {
        background-color: #ffb654;
    }
    .org {
        background-color: #49ace6;
    }
    .misc {
        background-color: #c2ccd1;
    }
    .loc {
        background-color: #3deb6c;
    }
</style>


## Résolution de toponymes et géocodage avec DBPedia

### Introduction à la résolution de toponymes 🔬

La **résolution de toponymes** entre dans le domaine de l'ingénierie des connaissances en tant que cas particulier d'une tâche générale nommée **liage d'entités** (en anglais *Entity Linking - EL*).
Le liage d'entités suit l'étape de reconnaissance des entités nommées (REN/*NER*) et consiste à associer les entités annotées avec une **ressource dans une base de connaissances de référence** avec deux objectifs :

1. **désambiguïser** les entités nommées, car on considère que deux entités nommées liées à la même ressource sont en fait deux mentions de la même entité du monde réel.
2. **enrichir** les informations extraites grâce aux métadonnées que fournit la base de connaissances. Dans le cas des lieux, on sera évidemment intéressé par leurs **coordonnées géographiques**.

La résolution de toponymes est tout simplement le nom donné au liage d'entités nommées qui désignent des lieux. Si, en plus, on récupère des coordonnées géographiques pour cartographier les toponymes, la tâche devient une tâche de **géocodage** !

La base de connaissance peut être n'importe quel référentiel externe, mais les **bonnes pratiques** du domaine veulent que l'on essaye généralement de privilégier soit des **référentiels d'autorité** comme le catalogue de la BnF, soit des grands **graphes de connaissances ouverts** et publiés sur le Web de données, par exemple Wikidata ou DBPedia qui sont des versions de Wikipedia sous forme de graphes de connaissances.

Le processus de résolution de toponymes peut être décomposé en trois parties  :

<img src="./img/schema.jpg" alt="Recherche dans un référentiel -> Désambiguisation -> Géocodage" width="80%"></img>

<span style="color: red; font-size: 1.2em;"><strong>🚨 Attention |</strong></span>  **Dans cet atelier nous négligeront l'étape de désambiguïsation.* Elle peut toutefois être essentielle selon le cas d'application.

### Échauffement

Commençons par la première étape de **recherche des entités nommées lieux (LOC)**.

Pour l'exemple, dans  l'extrait de l'article sur le naufrage du titanic on trouve deux mentions de lieux, <span class ="renbox loc">Staffordshire<sup>[LIEU]</sup></span> et <span class ="renbox loc">Liverpool<sup>[LIEU]</sup></span> : 


> « Le capitaine <span class ="renbox per">Smith<sup>[PERSONNE]</sup></span> qui commandait le <span class="renbox misc">Titanic<sup>[OBJET]</sup></span> est, nous l'avons dit hier, depuis plus de 35 ans au service de la <span class ="renbox org">White Star Line<sup>[ORGANISATION]</sup></span>.
> Il est actuellement agé de 60 ans.
>  Né dans le <span class ="renbox loc">Staffordshire<sup>[LIEU]</sup></span>, le capitaine <span class ="renbox per">Smith<sup>[PERSONNE]</sup></span> avait fait son apprentissage de marin dans la maison d'armement <span class ="renbox org">Gibson et C°<sup>[ORGANISATION]</sup></span>, de <span class ="renbox loc">Liverpool<sup>[LIEU]</sup></span>. »


Nous allons utiliser comme référentiel de [DBPediaFR](https://fr.dbpedia.org/) (https://fr.dbpedia.org/).
DBPediaFR est une représentation en graphe de connaissances du contenu de Wikipédia en français. 

Chaque page de Wikipédia en français devient donc une **ressource RDF** dans le graphe, qui possède une **URI** et qui est enrichie de nombreuses **métadonnées**.

Par exemple, la page Wikipédia de l'École Nationale des Chartes est transformée une ressources RDF dont l'URI (qui est ici un URL) est :
> http://fr.dbpedia.org/resource/École_nationale_des_chartes

Si l'on va à cet URL dans un navigateur Web, on peut voir un résumé de toutes les triplets RDF associés à cette ressource.

Mais comment trouve-t-on l'URI d'un lieu dans DBPediaFR à partir de son nom ?
Pour cela, on peut utiliser l'outil open source  [https://fr.dbpedia.org/lookup.html](https://fr.dbpedia.org/lookup.html), qui permet justement de faire cette recherche ! 

<div style="border-top: 1px solid #ff9800; padding: 10px; border-radius: 5px; color:#ff9800;"><strong>🧩 - QUESTION 1 - ⭐</strong></div>

En utilisant l'outil DBPediaFR lookup (https://fr.dbpedia.org/lookup.html), recherchez le lieu <span class ="renbox loc">Staffordshire<sup>[LIEU]</sup></span>.
Rendez-vous sur la page de présentation de la meilleure resource identifiée, et recherchez parmi les triplets associés s'il y a des coordonnées géographiques. 

Quel sont les noms et les préfixes RDF des propriétés utilisées pour décrire ces informations géographiques ? Notez les dans un coin, vous en aurez besoin plus tard !

<div style="border-bottom: 1px solid #ff9800; padding: 10px; border-radius: 5px; margin-top: -30px;"></div>

Vous venez de tester à la main la première étape de la  résolution de toponymes grâce à DBPedia Lookup.
Le but va maintenant être d'automatiser ce processus !


### Résoudre un toponyme avec l'API de DBPedia Lookup

[DPBedia Lookpup](https://github.com/dbpedia/dbpedia-lookup) va être très utile car son instance officielle [https://lookup.dbpedia.org/](https://fr.dbpedia.org/lookup.html) mets à disposition une API REST qui permet de l'interroger avec une simple URL.

Par exemple pour récupérer les meilleurs candidats pour le toponym *Staffordshire*, il suffit d'appeler : 


>https://fr.dbpedia.org/lookup/api/search?query=Staffordshire


In [ ]:
# Exécutez-moi ! 🚀

! curl -s "https://fr.dbpedia.org/lookup/api/search?query=Staffordshire" # -s pour le mode silencieux et ne pas afficher la progression de la requête

<span style="color: #40d6d1; font-size: 1.2em;"><strong>💡 Astuce |</strong></span> Une documentation est disponible ici : https://www.dbpedia.org/resources/lookup/

Vous aurez remarqué qu'on récupère les résultats au format XML, peu pratique à manipuler en Python.
Heureusement l'outil propose un mécanisme de **négociation de contenu** qui permet de préciser un format alternatif avec le paramètre `format=`...et il serait préférable de récupérer du JSON.

<div style="border-top: 1px solid #ff9800; padding: 10px; border-radius: 5px; color:#ff9800;"><strong>🧩 - QUESTION 2 - </strong></div>

Dans votre navigateur Web préféré, allez à l'URL suivante et vérifiez si les résultats renvoyés par l'API semblent correspondre à ce que vous obteniez avec la version graphique de DBPediaFR Lookup :

> https://fr.dbpedia.org/lookup/api/search?query=Staffordshire&format=JSON
 


In [ ]:
# Exécutez-moi ! 🚀

! curl -s "https://fr.dbpedia.org/lookup/api/search?query=Staffordshire&format=JSON" # -s pour le mode silencieux et ne pas afficher la progression de la requête


 <div style="border-bottom: 1px solid #ff9800; padding: 10px; border-radius: 5px; margin-top: -30px;"></div>

### Automatisation des appels à DBPedia Lookup

Bien sûr, on souhaite automatiser ces appels à l'API afin de pouvoir résoudre des toponymes reconnus dans un texte à la volée !

Maintenant, vous connaissez la bibliothèque `requests` !

In [ ]:
# %pip install -q requests

In [ ]:
import requests
url = "https://fr.dbpedia.org/lookup/api/search?query=Staffordshire&format=JSON"
response = requests.get(url)
#response.headers['Content-Type']
response.raise_for_status() # Appeler raise_for_status() après une requête GET est une bonne pratique : cela permet de lever automatiquement une exception si la requête a échoué.
print(type(response), response, response.headers['Content-Type'], response.text)
#print(response.text)

La fonction `dbpedia_lookup(toponyme: str)` ci-dessous prend en paramètre un `toponyme` sous forme de chaîne de caractère, et renvoie  le contenu de l'élément racine "docs" de la réponse HTTP sous forme d'une liste de ressources représentées par des dictionnaires Python.

In [ ]:
# Complétez-moi ! 🏗️

from typing import Any
from functools import cache


@cache
def dbpedia_lookup(toponyme: str) -> list[dict[str, Any]]:
    query = f"https://fr.dbpedia.org/lookup/api/search?query={toponyme}&format=JSON"
    response = requests.get(query)
    response.raise_for_status()

    response_json = response.json()
    docs = response_json.get("docs")
    return docs or []

### Récupération de l'URI de la meilleure resource identifiée

DBPediaFR nous renvoie la liste des resources correspondant à la requête. C'est bien, mais ce que l'on aimerait c'est accéder aux triplets associés à cette resource afin de récupérer les coordonnées géographiques associées, s'il y en a !

Nous allons donc avoir besoin d'un mécanisme supplémentaire pour :
1. **récupérer** les URI des resources dans les dictionnaires renvoyés par `dbpedia_lookup()`.
2. **filtrer la liste** pour conserver uniquement le meilleur résultat (=le premier) pour éviter des requêtes sur des resources que l'on utilisera pas.


Nous allons pour cela créer une nouvelle fonction qui englobera `dbpedia_lookup()`, nommées `dbpedia_top1(toponyme: str)-> str | None` qui prends en paramètre un toponyme et renvoie **l'URI de la meilleure resource identifiée par DBPediaFR lookup** (ou None s'il n'y a pas de résultat).


Chaque resources JSON renvoyée par l'API de DBPediaFR Lookup contient une propriété `resource` contenant une liste d'URI pour cette resource.
Nous considérerons que l'URI de la resource est **le premier élément de cette liste**.

Par exemple, ce sera "http://dbpedia.org/resource/Staffordshire" pour le toponyme "Staffordshire" :

In [ ]:
# Exécutez-moi ! 🚀
! curl -s "https://fr.dbpedia.org/lookup/api/search?query=Staffordshire&format=JSON" | jq ".docs[0].resource" # jq est un utilitaire bash pour manipuler du JSON, très très pratique !

La fonction `dbpedia_top1(str)` dans la cellule suivante récupère **l'URI de la meilleure resources trouvées pour un toponyme passé en argument**.
Si aucun résultat n'est trouvé, la fonction doit renvoyer None

In [ ]:
from functools import cache


@cache  # On active la mise en cache pour cette fonction, cf. l'astuce précédente
def dbpedia_top1(toponyme: str) -> str | None:
    r = dbpedia_lookup(toponyme)
    if not r:
        return None
    else:
        return r[0].get("resource")[0]

In [ ]:
# Exécutez-moi ! 🚀

t = "Staffordshire"
r = dbpedia_top1(t)

# La réponse doit être l'URI attendue, sous forme de chaîne de caractères
assert [isinstance(r, str), r == "http://f.dbpedia.org/resource/Staffordshire"]

t = "Cette ressource n'existe pas "
r = dbpedia_top1(t)

# Une resource inexistante doit retourner None, pas provoquer d'erreur
assert r is None

<div style="border-top: 1px solid #ff9800; padding: 10px; border-radius: 5px; color:#ff9800;"><strong>🧩 - QUESTION 5 - </strong></div>

Exécutez cette requête dans le *SPARQL endpoint* de DBPediaFR (https://fr.dbpedia.org/sparql) et vérifiez le résultat contient bien les latitude et longitude du Staffordshire.

<pre>
PREFIX geo: <http://www.w3.org/2003/01/geo/wgs84_pos#>
SELECT ?lat ?long
WHERE {<http://fr.dbpedia.org/resource/Staffordshire> (geo:lat) ?lat ; (geo:long) ?long.}
</pre>

<span style="color: #40d6d1; font-size: 1.2em;"><strong>💡 Astuce |</strong></span> Ces coordonnées sont celles d'un point, alors que Stafforshire est un comté; c'est donc une description géographique grossière d'une entité surfacique, sans doute son centroïde. Vous pouvez d'ailleurs copier ces coordonnées dans [Google Maps](https://www.google.fr/maps) pour voir quel emplacement sur Terre elles désignent.

 <div style="border-bottom: 1px solid #ff9800; padding: 10px; border-radius: 5px; margin-top: -30px;"></div>


In [ ]:
# Exécutez-moi ! 🚀

! curl -s -X POST \
-H "Content-Type: application/sparql-query" \
-H "Accept: application/json" \
--data '''PREFIX geo: <http://www.w3.org/2003/01/geo/wgs84_pos#> SELECT ?lat ?long WHERE { <http://fr.dbpedia.org/resource/Staffordshire> (geo:lat) ?lat ; (geo:long) ?long. }''' \
  http://fr.dbpedia.org/sparql \
  | jq #On passe le résultat à jq pour le formater proprement

Reste donc à transformer cela en fonction Python 🙂

<span style="color: #40d6d1; font-size: 1.2em;"><strong>💡 Astuce |</strong></span> Dans le résultat de la cellule précédente, on voit que les valeurs des coordonnées sont relativement "profond" dans l'objet JSON retourné, appellons le `résultat`. Pour accéder à la latitude, il faut par exemple regarder dans "results", puis "bindings", puis "lat" puis "value".  On a donc envie d'écrire `résultat["results"]["bindings"]["lat"]["value"]`. Oui mais voilà : si n'importe laquelle des clés n'existe pas, le code va planter ! On pourrait gérer ça avec des clauses imbriquées `if "clé" in résultats["..."]: ...`, mais c'est très lourd.
Mais il existe une astuce en utilisant la méthode `résultat.get("clé")` à la place de `résultat["clé"]`. Ces deux formes d'accès au contenu d'un dictionnaires sont presques similaires, sauf que :
- `résultat.get("clé")` renvoie `None` si "clé" n'est pas une clé du dictionnaire, alors que dans ce cas `résultat["clé"]` lèvera une exception ;
- `résultat.get()` accepte un second paramètre qui est la valeur retournée par défaut **si la clé n'existe pas**.

L'astuce est alors de retourner un dictionnaire vide dans ce cas, ce qui permet de chainer les appels à `résultat.get` pour parcourir les éléments imbriqués : 

In [ ]:
# Complétez-moi ! 🏗️


@cache  # Mise en cache, les requêtes SPARQL sont coûteuses en temps !
def geocode(uri: str) -> tuple[float, float]:

    # Le patron de requête SPARQL: l'URI sera substituée à la place de %s par l'argument de la fonction
    sparql_template = """PREFIX geo: <http://www.w3.org/2003/01/geo/wgs84_pos#> SELECT ?lat ?long WHERE {<%s> (geo:lat) ?lat ; (geo:long) ?long. }"""

    # On substitue l'URI dans le template de requête SPARQL en utilisant le formatage ancien de Python plutôt que les f-strings car la requête SPARQL contient déjà des accolades.
    sparql_query = sparql_template % uri

    http_headers = {
        "Content-Type": "application/sparql-query",
        "Accept": "application/json",
    }
    response = requests.post(
        "http://fr.dbpedia.org/sparql", headers=http_headers, data=sparql_query
    )
    response.raise_for_status()
    json_response = response.json()

    # Récupérer l'élément results -> bindings dans l'objet json_response
    # ⚠️ ATTENTION : l'élement "bindings" est une liste, donc l'argument par défaut pour get("bindings") doit être une liste vide => get("bindings", [])
    bindings = json_response.get("results", {}).get("bindings", [])

    # Si la liste "bindings" est vide, lat et long seront None
    if not bindings:
        return None, None

    # Sinon, le premier élément de la liste "bindings" est un dictionnaire contenant les valeurs de lat et long
    bindings = bindings[0]

    # Récupérer la valeur de latitude dans : lat -> value
    lat = bindings.get("lat", {}).get("value", None)

    # Récupérer la valeur de longitude dans : long -> value
    long = bindings.get("long", {}).get("value", None)

    # lat et long sont récupérés en tant que chaînes de caractères, on les convertit en float
    lat = float(lat) or None
    long = float(long) or None

    return lat, long

In [ ]:
# Exécutez-moi !   🚀

uri = "http://fr.dbpedia.org/resource/Staffordshire"

coordonnees = geocode(uri)
print(f"{uri=} => {coordonnees=}")

assert all(
    [
        isinstance(coordonnees, tuple),
        len(coordonnees) == 2,
        all(isinstance(x, float) for x in coordonnees),
        coordonnees == (52.8333, -2.0),
    ]
)

In [ ]:
def get_coordinates(toponyme: str) -> tuple[float, float]:
    uri = dbpedia_top1(toponyme)
    if not uri:
        lat, long = None, None
    else:
        lat, long = geocode(uri)
    return lat, long

In [ ]:
def get_coordinates_verbose(toponyme: str) -> tuple[float, float]:

    uri = dbpedia_top1(toponyme)
    print(f"{toponyme=} => {uri=}", end="")

    if not uri:
        lat, long = None, None
    else:
        lat, long = geocode(uri)

    print(f" => {lat=}, {long=}")
    return lat, long

 <div style="border-bottom: 1px solid #ff9800; padding: 10px; border-radius: 5px; margin-top: -30px;"></div>

## Reconnaître automatiquement et cartographier les lieux d'un texte

<span style="color: #85d0ff; font-size: 1.2em;"><strong>ℹ️ Info |</strong></span> Cette seconde partie ne contient pas de question mais est un tutorial pour appliquer la chaîne de traitement construire à des textes dont les lieux sont reconnus à l'aide du modèle SpaCy entraîné dans la partie 1.


### Premier essai


L'idée est maintenant de pouvoir appliquer la chaîne de traitement à toutes les entités LOC extraites d'un texte historique par le modèles SpaCy entraîné dans la partie 1 !


Commençons par charger le modèle, qui devrait se trouver dans le dossier `../partie_1/ner_presse_ancienne/model-best`.

In [ ]:
import spacy

nlp = spacy.load("ner/model-best")

In [ ]:
# Exécutez-moi ! 🚀
# Même traitement que précédemment mais avec un meilleur modèle

from IPython import display
from spacy import displacy

with open("docs/bpt6k65591428.txt", "r") as file:
    texte = file.read()

doc = nlp(texte)
display.HTML(displacy.render(doc, style="ent", jupyter=False, page=True))


Le modèle prédit des entités de différentes classes (ORG, PER, LOC), mais nous ne sommes intéressés que par les lieux (LOC).
Filtrons donc les entités nommées reconnues pour ne ressortir que les lieux.

In [ ]:
loc = [ent.text for ent in doc.ents if ent.label_ == "LOC"]
loc

Nous pouvons maintenant exécuter la chaîne de traitement de résolution de toponymes pour chaque entité nommée LOC !

In [ ]:
# Exécutez-moi ! 🚀

for toponyme in loc:
    get_coordinates_verbose(toponyme)

### Géocodage d'un texte

Nous voici capables de récupérer les coordonnées des lieux dans un texte grâce à SpaCY et notre chaîne de traitement !

Afin de réutiliser le mécanisme complet, créons une fonction `geocode_texte(texte: str, model: spacy.language.Language) -> tuple[list, list]` qui :
- prend en argument un texte quelconque et le modèle SpaCy
- renvoie deux listes : une liste de toutes les mentions de lieux reconnus, et une liste des paires de coordonnées identifiées pour chacun

In [ ]:
# Exécutez-moi ! 🚀
def geocode_texte(
    text: str, nlp: spacy.language.Language
) -> tuple[list[str], list[tuple[float, float]]]:
    doc = nlp(text)
    loc = [ent.text for ent in doc.ents if ent.label_ == "LOC"]
    coordonnees = [get_coordinates(toponyme) for toponyme in loc]
    return loc, coordonnees

In [ ]:
%pip install pandas
import pandas as pd

In [ ]:
from collections import Counter
pd.set_option('display.max_rows', 200)

loc_dict = dict(Counter(loc)) # dédoublonner la liste des lieux avec un compteur, dans un dict
# ajouter les coordonnées de chaque lieu dans le dict
for k,v in loc_dict.items():
    loc_dict[k]=[loc_dict[k], dbpedia_top1(k), get_coordinates(k)]
# ajouter url et coordonnées pour chaque lieu
df = pd.DataFrame.from_dict(loc_dict, orient='index',
                           columns=['occs', 'db_pedia', 'coordinates'])
df

## Cartographie avec Folium


Folium est une bibliothèque Python pour créer des cartes interactives extrêmement facilement; nous allons l'utiliser pour afficher la densité des mentions de toponymes localisés sous la forme d'une **carte de chaleur interactive** !

Commençons par installer puis importer Folium .

In [ ]:
# Exécutez-moi ! 🚀

%pip install -q folium

import folium

In [ ]:
# Exécutez-moi ! 🚀

folium_map = folium.Map(
    location=[48.8566, 2.3522],  # On précise les coordonnées du centre de la carte...
    zoom_start=3,  # ... et le niveau de zoom initial. 0 = vue du monde, 18 = vue très rapprochée
)

# On affiche l'objet Map
folium_map

Ajouter des couches cartographiques est également très facile.

Pour créer une carte de chaleur, il suffit de créer un objet de type `HeatMap`, disponible dans le module `folium.plugins`.

L'objectif est donc de créer une carte de chaleur à partir des lieux géocodées dans un texte grâce à  `geocode_texte()`.

Le constructeur de `HeatMap` accepte une liste de coordonnées géographiques **valides**. Nous devons donc préalablement retirer les paires de coordnnées `(None, None)` potentiellement produites par la chaîne de traitement lorsque des toponymes ne sont pas résolus.


In [ ]:
print(coordonnees)

In [ ]:
# Exécutez-moi ! 🚀

coordonnees_valides = [(lat, long) for lat, long in coordonnees if lat and long]

print("Coordonnées valides :", len(coordonnees_valides), "/", len(coordonnees))

Il ne reste plus qu'à créer un objet `HeatMap` à partir de ces coordonnées valides, puis l'ajouter à la carte !

In [ ]:
# Exécutez-moi ! 🚀

from folium.plugins import HeatMap

# On crée une carte de chaleur à partir des coordonnées valides
heatmap = HeatMap(coordonnees_valides)

# On ajoute la carte de chaleur à la carte folium
heatmap.add_to(folium_map)

Ce qui permet de construire la projection cartographique finale des mentions de lieux identifiés dans le texte de presse :

In [ ]:
# Exécutez-moi ! 🚀

folium_map

## Une App Dash pour tester le code

Si vous voulez expérimenter d'avantage, **lancez l'application Dash dashboard.py** qui réutilise le code créé ici et permet de projeter sur une carte un texte récupéré depuis Gallica grâce à son [API Document](https://api.bnf.fr/fr/api-document-de-gallica) ! 

Dans cette application, on peut rapidement comparer les cartes générées pour 2 publications successives :

- https://gallica.bnf.fr/ark:/12148/bpt6k65591428 => ICOM 1964 (première mention du Festival mondial des arts nègres)
- https://gallica.bnf.fr/ark:/12148/bpt6k6567411d => ICOM 1968 (nouvelles mention du Festival et pour comparaison des cartes mondiales)


In [ ]:
# Exécutez-moi ! 🚀
# ! conda install -c conda-forge dash-bootstrap-components
! python3 dashboard.py

In [ ]:
import urllib.parse
url = 'https://gallica.bnf.fr/SRU?version=1.2&operation=searchRetrieve&query=gallica%20all%20%22FESTAC%22%20sortby%20ocr.quality/sort.descending&filter=dc.type%20all%20%22fascicule%22%20and%20dc.language%20all%20%22fre%22%20and%20dc.date%20%3E=%20%221976%22'
urllib.parse.unquote(url)